In [ ]:
!pip install transformers datasets pandas scikit-learn matplotlib seaborn wordcloud arabert


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 10.8 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186456 sha256=31602ff9be3bf64b7bec07b0020a57bd911cfe8654cae7127b4314ef77be44a8
  Stored in directory: /root/.cache/pip/wheels/94/08/b4/78657b1541bb704b088317b52429ee4016d9888fe47dbb130f
Successfully built emoji


In [ ]:
!apt-get update
!apt-get install -y fonts-dejavu-core fonts-dejavu-extra
# install specific font family like arial (ttf)
!apt-get install -y ttf-mscorefonts-installer
!fc-cache -fv

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,759 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,740 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,984 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/u

In [ ]:
!sudo apt install -y libfreetype6-dev
!pip install --upgrade Pillow
!pip install arabert

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libfreetype6-dev is already the newest version (2.11.1+dfsg-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
import pandas as pd
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from arabert.preprocess import ArabertPreprocessor
# Import the font manager
import matplotlib.font_manager as fm

# Charger le dataset
df_combined = pd.read_csv("/content/donnees_emotions.csv")  # pour darija marocaine
# df_arq = pd.read_csv("arq.csv")  # pour dialecte algérien

# Colonnes de labels (les mêmes pour les deux datasets)
LABELS = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]

# Supprimer les stop phrases
STOP_PHRASES = ["@<username>", "الراوي:", "واحد الراجل", "بين زوج بنات"]

def clean_text(text):
    text = text.lower()
    for phrase in STOP_PHRASES:
        text = text.replace(phrase, "")
    text = re.sub(r"http\S+|www\S+|@\S+|#\S+", "", text)  # remove URLs, mentions, hashtags
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
    return text.strip()

# Nettoyage
df_combined["clean_text"] = df_combined["text"].apply(clean_text)
# df_arq["clean_text"] = df_arq["text"].apply(clean_text)

# Wordcloud avant/après nettoyage
def plot_wordcloud(texts, title):
    text = " ".join(texts)

    # Find the path to the Arial font using Matplotlib's font manager
    font_path = fm.findfont("Arial")  # Define font_path here

    wc = WordCloud(font_path=font_path, background_color='white', width=800, height=400).generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wc, interpolation='bilinear')
    plt.title(title)
    plt.axis("off")
    plt.show()

# plot_wordcloud(df_ary["clean_text"], "ARY - Après nettoyage")
# plot_wordcloud(df_arq["clean_text"], "ARQ - Après nettoyage")

## Combiner les datasets

In [ ]:
# # Vérifie que les colonnes LABELS sont bien présentes dans les deux
# df_ary[LABELS] = df_ary[LABELS].astype(int)
# df_arq[LABELS] = df_arq[LABELS].astype(int)

# # Fusion des datasets
# df_combined = pd.concat([df_ary, df_arq], ignore_index=True)


In [ ]:
# MARBERT v2
model_name = "UBC-NLP/MARBERTv2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenisation
def tokenize_data(df):
    return tokenizer(
        df["clean_text"].tolist(),
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
# !pip install --upgrade transformers

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
# from transformers import Trainer, TrainingArguments
# Import PreTrainedModel and AutoModelForSequenceClassification from modeling_utils
# from transformers.modeling_utils import PreTrainedModel, AutoModelForSequenceClassification
from sklearn.metrics import f1_score, hamming_loss
import numpy as np

# Préparation des labels
LABELS = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]
df_combined[LABELS] = df_combined[LABELS].astype(int)

class EmotionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels.values, dtype=torch.float)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

# Split
train_df, val_df = train_test_split(df_combined, test_size=0.1, random_state=42)
train_enc = tokenize_data(train_df)
val_enc = tokenize_data(val_df)

train_dataset = EmotionDataset(train_enc, train_df[LABELS])
val_dataset = EmotionDataset(val_enc, val_df[LABELS])

# Charger modèle
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(LABELS), problem_type="multi_label_classification")

# Fonction de metrics
def compute_metrics(pred):
    logits, labels = pred
    preds = (logits >= 0.5).astype(int)
    return {
        'f1': f1_score(labels, preds, average='micro'),
        'macro_f1': f1_score(labels, preds, average='macro'),
        'hamming_loss': hamming_loss(labels, preds)
    }

# Entraînement
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=12,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


config.json:   0%|          | 0.00/757 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/MARBERTv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/654M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,F1,Macro F1,Hamming Loss,Runtime,Samples Per Second,Steps Per Second
1,No log,0.423914,0.214173,0.175560,0.195227,1.328500,320.653000,13.549000
2,No log,0.389180,0.441919,0.400011,0.172926,1.388800,306.742000,12.961000
3,No log,0.404466,0.524444,0.470829,0.167449,1.360400,313.152000,13.232000
4,0.361500,0.437514,0.506944,0.477038,0.166667,1.371200,310.688000,13.128000
5,0.361500,0.468070,0.526316,0.508411,0.165493,1.368000,311.394000,13.158000
6,0.361500,0.512229,0.569373,0.542924,0.163928,1.359200,313.416000,13.243000
7,0.118700,0.556554,0.563636,0.545578,0.169014,1.365400,311.987000,13.183000
8,0.118700,0.586354,0.560496,0.541081,0.166275,1.369900,310.981000,13.140000
9,0.118700,0.605224,0.561915,0.539039,0.164710,1.365500,311.983000,13.182000
10,0.031900,0.630946,0.576098,0.552025,0.162363,1.358600,313.563000,13.249000


TrainOutput(global_step=1920, training_loss=0.13601857994993527, metrics={'train_runtime': 1044.3967, 'train_samples_per_second': 44.029, 'train_steps_per_second': 1.838, 'total_flos': 2481308585723520.0, 'train_loss': 0.13601857994993527, 'epoch': 12.0})

## Sauvegarder le modèle

In [ ]:
from transformers import AutoTokenizer

# Sauvegarder le modèle et le tokenizer
trainer.save_model("/content/mon_modele_marbbert2")  # dossier de sauvegarde
tokenizer.save_pretrained("/content/mon_modele_marbbert2")

('/content/mon_modele_marbbert2/tokenizer_config.json',
 '/content/mon_modele_marbbert2/special_tokens_map.json',
 '/content/mon_modele_marbbert2/vocab.txt',
 '/content/mon_modele_marbbert2/added_tokens.json',
 '/content/mon_modele_marbbert2/tokenizer.json')

In [ ]:
import shutil

# Chemin du dossier à zipper et du fichier zip de sortie
dossier_a_zipper = "/content/mon_modele_marbbert2"
fichier_zip = "/content/mon_modele_marbbert2.zip"

# Création du fichier zip
shutil.make_archive(fichier_zip.replace(".zip", ""), 'zip', dossier_a_zipper)


'/content/mon_modele_marbbert2.zip'

### Charger plus tard le modèle pour l’inférence

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("/content/mon_modele_marbbert2")
tokenizer = AutoTokenizer.from_pretrained("/content/mon_modele_marbbert2")
model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
# Phrase(s) à tester
# sentences = [
#     "أنا زعفان بزاف من الخدمة ديالي",  # darija
#     "كوثر براني خرجات من دارها من بعد 20 يوم ديال الحجر الصحي باش دير الجلبة لبنتها وكتعيش خوف مضاعف",
#     "وش بيك اليوم؟ راك فرحان!",       # algérien
# ]

sentences = [
    "أنا فرحان بزاف اليوم، الحمد لله",
    "أنا خايف نضيع الفرصة ديال حياتي",
    "قلبي راه محروق على اللي صرا",
    "ضحكت بزاف كي شفت الفيديو تاعك!",
    "خفت كي سمعت الصوت في الليل.",
]

# Nettoyage
clean_sentences = [clean_text(s) for s in sentences]

# Tokenisation
inputs = tokenizer(
    clean_sentences,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

# Prédiction
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    probs = torch.sigmoid(outputs.logits).numpy()  # sigmoid pour multi-label
    preds = (probs >= 0.5).astype(int)  # seuil 0.5

# Affichage
for i, sentence in enumerate(sentences):
    print(f"\nTexte : {sentence}")
    print("Émotions prédites :")
    for label, pred in zip(LABELS, preds[i]):
        if pred:
            print(f" - {label}")



Texte : أنا فرحان بزاف اليوم، الحمد لله
Émotions prédites :
 - joy

Texte : أنا خايف نضيع الفرصة ديال حياتي
Émotions prédites :
 - sadness

Texte : قلبي راه محروق على اللي صرا
Émotions prédites :
 - sadness

Texte : ضحكت بزاف كي شفت الفيديو تاعك!
Émotions prédites :
 - joy

Texte : خفت كي سمعت الصوت في الليل.
Émotions prédites :
 - fear


In [ ]:
 !pip install gradio


In [ ]:
# Fonction de prédiction mise à jour
def predict(text):
    cleaned = clean_text(text)
    inputs = tokenizer(cleaned, return_tensors="pt", truncation=True, padding=True, max_length=128)
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.sigmoid(outputs.logits).numpy()[0]
        preds = (probs >= 0.5).astype(int)

    # Filtrer les émotions détectées (valeur = 1)
    detected_emotions = [label for label, pred in zip(LABELS, preds) if pred == 1]

    # Retourner soit la liste, soit un message clair
    return detected_emotions if detected_emotions else ["Aucune émotion détectée"]


In [ ]:
import gradio as gr

interface = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(lines=3, placeholder="Écris une phrase en darija ou algérien..."),
    outputs="json",
    title="🧠 Analyse des émotions - Darija & Arabe Algérien",
    description="Ce modèle prédit les émotions présentes dans les phrases en dialectes arabes (marocain ou algérien)."
)

interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6c69c2a265f49fbf59.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
